# Taller Fourier EDM - Dispositivo vestible

## Práctica 2 - Señales de Movimiento  (HAR - Human activity recognition)

### Actividades
1. Búsqueda bibliográfica
2. Visualización de señales
3. Análisis de señales en el tiempo. 
4. Análisis de señales en frecuencia. Transformada de Fourier, Espectrograma. Armónicos
5. Manejo de datos por ventanas de observación. Cálculo de características de movimiento. 
6. Identificación de agrupamiento de actividades humanas en el espacio de características

------




## Base de señales de movimiento

Usaremos la base de datos "Human-Activity-Recognition Using Smartphones Data Set" descrita en [1] y disponible en [2].  
> __NOTA:__ En nuestro Taller, usaremos una versión adaptada de los datos que denominaremos base __HAR__ y está disponible en el directorio __data__. No es necesario bajar la base original.


Este conjunto de datos se recopiló de 30 personas (denominadas "sujetos") que realizaban diferentes actividades con un teléfono inteligente en la cintura. Los datos se registraron mediante sensores (acelerómetro y giroscopio) en dicho teléfono. El experimento se grabó también en video para etiquetar los datos manualmente.
Los datos de acelerómetro y giroscopio fueron adquiridos a una frecuencia de muestreo de 50 Hz.


### Señales de aceleración y giro

Los datos se encuentran en la carpeta __data/UCI_HAR__.
Estos datos difieren levemente de los originales de la base. Han sido adaptados para este notebook.


Cada archivo en formato CSV tiene las siguientes columnas:     
* [acc_x]  aceleración en el eje X
* [acc_y]  aceleración en el eje Y
* [acc_z]  aceleración en el eje Z
* [gyro_x] velocidad angular en torno al eje X
* [gyro_y] velocidad angular en torno al eje Y
* [gyro_z] velocidad angular en torno al eje Z
* [label] etiqueta de actividad
* [subject_id] identificador del sujeto realizando la actividad
* [activity_name] nombre de la actividad

Los datos de aceleración están expresados en unidades de "g". 
Los datos de velocidad angular están expresados en rad/s

### Etiquetas

En el conjunto de datos, las etiquetas se representan con números del 1 al 6 como identificadores.

- CAMINANDO como __1__
- SUBIENDO ESCALERAS como __2__
- BAJANDO ESCALERAS como __3__
- SENTADO como __4__
- DE PIE como __5__
- ACOSTADO como __6__

### Partición de los datos 
- Las lecturas del ___70%___ de los voluntarios se tomaron como ___datos de entrenamiento___ y las del ___30%___ restante se registraron como ___datos de prueba___.

------

[1] Anguita, D., Alessandro Ghio, L. Oneto, Xavier Parra and Jorge Luis Reyes-Ortiz. “A Public Domain Dataset for Human Activity Recognition using Smartphones” The European Symposium on Artificial Neural Networks (2013).  
  
[2] Reyes-Ortiz, Jorge, Davide Anguita, Alessandro Ghio, Luca Oneto, and Xavier Parra. 2013. Human Activity Recognition Using Smartphones. UCI Machine Learning Repository. https://doi.org/10.24432/C54S4K. (Accedido 27/7/2025)

--------

### Notación de consignas

A lo largo del notebook encontrarán las siguientes palabras claves:

* **INVESTIGAR**  -> indica que se requiere investigar un tema y hacer un breve reporte. 


* **COMPLETAR**  -> indica que se requiere completar el código. 
  
  
* **EXPERIMENTAR** -> indica que la celda contiene código funcional que permite experimentar. Es posible, si lo requiere la experimentación duplicar la celda y realizar los cambio que sean de interés en el código.
  
  
* **DISCUTIR** -> se espera una discusión de los experimentos realizados y/o de las preguntas formuladas
  
  
  
* **RESPONDER** -> se espera una respuesta a preguntas concretas

-----
-----

## 0. Investigación bibliográfica



**INVESTIGAR**  
Haremos primero una breve revisión del tema de reconocimiento de actividad mediante sensores de movimiento. Partiendo del artículo “A Public Domain Dataset for Human Activity Recognition using Smartphones”, la idea es armarse un panorama del área viendo qué otros artículos relevantes hay del tema, qué datos usan y cómo enfrentan el problema de reconocimiento. Se realizará un informe de 2 a 4 carillas en documento pdf aparte.

Podriamos pedirle esto a alguna AI, obviamente, pero también vamos a usar algunas otras herramientas que están disponibles y es bueno tenerlas a mano.

**Herramientas a explorar**
1. Google scholar:  https://scholar.google.com.uy/  
* Introducir el artículo "A public domain ..." y realizar la búsqueda.
* Ver cuántas citas tiene. Ver los artículos que lo citan. Se puede acotar una nueva búsqueda dentro de estos artículos.
* Probar filtros de búsqueda por fecha. Ver más opciones en "Búsqueda avanzada"
* ¿Hay dentro de los artículos relacionados alguno que sea una revisión del área y nos ayude a ver el panorama general? Estos artículos tienen en su título en general alguna palabra como "review", "survey" o  "trends in".

2. Grafos de artículos conectados  
Estas herramientas permiten encontrar los artículos relacionados como en Scholar pero permiten visualizarlos en forma de grafos.  
    a. Research Rabbit:  https://researchrabbitapp.com/    
    b. Connected papers:  https://www.connectedpapers.com/  (de pago luego de los 5 grafos)

3. Organizadores de bibliografía   
En la búsqueda bibliográfica, van a bajar algunos artículos, hacer anotaciones, citar en el informe. Para organizar esa información es conveniente tener algún  manejador de referencias.  
Ver por ejemplo Zotero: https://www.zotero.org/support/quick_start_guide

4.  Timbó  
Algunos artículos no están disponibles libremente, pero en Uruguay tenemos acceso a muchas colecciones mendiante el portal Timbó. Requiere un usuario que se hace con el número de cédula.  
https://timbo.org.uy/home  
    * Ver las colecciones disponibles 
    * Ver Scopus que puede ser de utilidad para el informe




















Definiciones generales de algunos módulos a utilizar

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.fft import fft, fftshift, ifft
from scipy.signal import spectrogram
import matplotlib.pyplot as plt


#estilo de las gráficas
plt.style.use('ggplot')

# autoreload. Los cambios en modulos externos como 'funciones_practica_1.py' se recargan automaticamente
%load_ext autoreload
%autoreload 2

# FORMAS DE VER LAS GRAFICAS --------------------
# ELEGIR UNA DE LAS OPCIONES Y DES-COMENTAR (sacar # de la linea)
# ----------------
# a) graficas en línea  entre las celdas (no interactivo)
%matplotlib inline
# ---------------- 
# b) graficas en línea  entre las celdas (interactivo, si se usa jupyter notebook) 
# %matplotlib notebook
# ----------------
# c) graficas en ventanas externas (abre una ventana por cada figura)
# %matplotlib qt
# ----------------
# d) Si se usa "jupyter lab" en lugar de "jupyter notebook" usar %matplotlib widget en lugar de %matplotlib notebook 
#    Si se usa vscode usar también %matplotlib widget en lugar de %matplotlib notebook 
# requiere instalar el modulo "ipympl". Ver https://stackoverflow.com/questions/51922480/javascript-error-ipython-is-not-defined-in-jupyterlab#56416229
# %matplotlib widget
#---------------------------------------------------


## 1. Carga e inspección de los datos

### 1.1  Cargar los datos de train a un DataFrame de Pandas

En este notebook se va a trabajar con los datos de __train__ de la base HAR que están disponibles en  __data/har_train.csv__.


#### 1.1.1 Cargar los datos a un DataFrame de Pandas.

In [ ]:
# COMPLETAR: Cargar los datos de train a un DataFrame de Pandas
df_har = .....

In [ ]:
# COMPLETAR Inspeccionar los datos cargados. Ver columnas, tipos de datos, etc.



#### 1.1.2 Explicar cómo están organizados los datos  
* Qué son los datos en cada fila ?  
* Qué son los datos en cada columna ?

**RESPONDER**

Respuesta:

#### 1.1.3 Análisis de la composición de los datos

En esta parte se busca entender la composición de los datos. Entre otros, se quiere entender:  
* Cuántas clases de actividades hay en los datos? Cuáles son esas clases ?
* Cuántos sujetos hay en la base? Cuáles son sus identificadores ?
* Cuántas muestras hay por cada sujeto? 
* Cuántas muestras hay por cada clase? Todas las clases están igualmente representadas en los datos ?



In [ ]:
# COMPLETAR  
# ALgunas de las preguntas se pueden responder usando funciones de Pandas como:
# unique(), value_counts(), etc.
# Se puede hacer también agrupamientos mediante:
# groupby()




In [ ]:
# COMPLETAR: Hacer un gráfico de barras con la cantidad de muestras por cada clase



In [ ]:
# COMPLETAR: Hacer un gráfico de barras con la cantidad de muestras por sujeto

              


## 2. Visualización de los datos 

### 2.1 Graficar los datos para un sujeto

Graficar las series temporales de los 3 ejes de acelerómetro para un sujeto

* a) Mostrar las 3 series temporales en la misma gráfica
* b) Agregar la información de etiqueta en la misma gráfica o en subplot de manera que puedan verse señales y etiquetas en conjunto

> **Importante:** Ya sea que trabajen en un jupyter notebook o en un entorno de desarrollo, es imprescindible para esta práctica, que puedan hacer zoom en las gráficas. Si no pueden lograr eso con el paquete de gráficos matplotlib, pueden usar el paquete de gráficos plotly  

In [ ]:
# COMPLETAR: Graficar las series temporales de los 3 ejes de acelerómetro para un sujeto
# Seleccionar un sujeto específico para graficar
# (puede ser el sujeto 1, pero se puede elegir cualquier otro sujeto)
subject_id = 1  

subject_data = df_har[df_har['subject_id'] == subject_id]

# COMPLETAR


### 2.2 Inspección de las series temporales

#### 2.2.1 Funcionamiento de los acelerómetros
Explique el principio de funcionamiento de un acelerómetro

**CONTESTAR**  

Respuesta:



#### 2.2.2 Orientación de los acelerómetros

Mirando la gráfica de datos de acelerómetro del sujeto:
  * Qué diferencias observa de niveles de señal entre los acelerómetros  X, Y y Z  ?
  * A qué se deben esas diferencias ?  
  * Puede determinar la orientación de los ejes respecto del cuerpo con la que se adquirieron los datos ?

**CONTESTAR**  

Respuesta:

#### 2.2.3 Walking 

En la gráfica del sujeto, haga zoom en una sección de la clase "WALKING"

* Ve algún patrón periódico en la señal ? 
* Cuál es el período de ese patrón ? Es igual en todos los ejes ? Hay algún desfasaje entre los ejes ?


**CONTESTAR**  

Respuesta:

Considere un fragmento de 4 segundos de algún período de caminar  
  * Busque relacionar cada parte de las señales de aceleración con el ciclo de marcha de una persona  
  * Puede también estudiar esto capturando señales de marcha con su celular 
    * Instale una aplicación que le permita capturar las señales del acelerómetro del celular (ej. [Phyphox](https://phyphox.org/) pero hay varias más)
    * Identifique los ejes y busque recrear la disposición del celular para obtener señales similares a las de la base HAR
    * Capture señales de marcha
    * Exporte las señales a archivos CSV para su posterior análisis
    * Opcionalmente se puede adquirir simultáneamente un video de la marcha para estudiar la secuencia con un software de análisis de movimiento (ej. [Kinovea](https://www.kinovea.org/)) 

**DISCUTIR**  los resultados del análisis de la marcha.  
Mostrar  capturas de aceleración realizadas con el celular, el análisis de marcha sobre las mismas y anotaciones de las condiciones de adquisición.  

Resultados:

## 3. Análisis en frecuencia

### 3.1 Transformada de Fourier 

#### 3.1.1 Transformada de la serie temporal completa 
Muestre la transformada de Fourier de las señales de aceleración completas de los 3 ejes del sujeto seleccionado 

In [ ]:
# COMPLETAR  Gráficas de las transformadas de las señales completas para los 3 ejes del sujeto seleccionado




#### 3.1.2 Transformada de la serie temporal por actividad 
Muestre la transformada de Fourier de las señales de aceleración  de los 3 ejes del sujeto seleccionado para la actividad de "WALKING" y de "WALKING DOWNSTAIRS" 

In [ ]:
# COMPLETAR Gráficas de las transformadas por actividad WALKING

In [ ]:
# COMPLETAR Gráficas de las transformadas por actividad WALKING_DOWNSTAIRS

#### 3.1.3 Preguntas

* a) Tiene sentido hacer la transformada de la serie temporal completa de un sujeto ?
* b) En esta base tenemos las etiquetas de actividad y pudimos aislar una actividad concreta. En general es una información que no tendremos. ¿Cómo podríamos analizar las componentes de frecuencia de una serie temporal con múltiples actividades que se suceden en el tiempo?


* c) Para la transformada de la actividad de caminar,
    * Compare las componentes de frecuencia con lo encontrado en el análisis de la marcha hecho sobre la señal en 2.2.3
    * Cómo explica las diferencias de componentes entre ejes ?



**CONTESTAR**  

Respuestas:



### 3.2 Espectrogramas


#### 3.2.1 ¿Qué es un espectrograma y cómo se calcula?

* a) Explicar qué es y cómo se calcula el espectrograma de una señal
* b) Buscar la función **scipy.signal.spectrogram** y explicar los parámetros de la misma __fs__, __nperseg__, __noverlap__, y __window__ 
* c) Para las series temporales completas de un sujeto, explicar qué valores de parámetros a usar en la función __spectrogram__  le parecen adecuados y por qué.


**CONTESTAR**  

Respuestas:



#### 3.2.2 Desplegar espectrogramas

Muestre los espectrogramas de las señales de acelerómetro para un sujeto

In [ ]:
# COMPLETAR
subject_id = 1  #valor a definir

window_size = 10   #valor a definir
window_overlap = 10 #valor a definir
window_name = 'ventana'  #valor a definir
fs = 10     #valor a definir

# Seleccionar un sujeto de la base

# Mostrar en subplots, las señales de aceleración y los espectrogramas  para los 3 ejes del sujeto seleccionado




#### 3.2.3 Analizar los espectrogramas

Revisar los gráficos obtenidos 

* Cómo se relaciona con lo obtenido en la sección 3.1 ?
* Fue buena la elección de parámetros para el cálculo de los espectrogramas ?



**CONTESTAR**  

Respuestas:



## 4. Filtrado de datos

### Datos de aceleración total y corporal
Los datos del acelerómetro en la base muestran la aceleración constante de la gravedad (1g en la dirección vertical) sumada a la aceleración propia del cuerpo del sujeto.  
Si es de interés tener sólo la aceleración debida al movimiento del cuerpo, podemos filtrar la aceleración constante aplicando un filtrado pasa-altos con una baja frecuencia de corte. Alternativamente podemos aplicar un filtro pasa-bajos y restar la parte de baja frecuencia a nuestra señal total.





### 4.1 Diseñar un filtro de Butterworth pasa-altos

#### 4.1.1  Diseño del filtro  
Tipo: pasa-altos  
Frecuencia de corte: 0.2 Hz  
Orden del filtro: 2

Obtener los coeficientes, mostrar ceros y polos, mostrar respuesta en frecuencia.  
Ver funciones de procesamiento de señales como __butter__, __freqz__, __tf2zpk__  en:  
[Signal processing (scipy.signal)](https://docs.scipy.org/doc/scipy-1.16.1/reference/signal.html)  


> Nota: Para un diseño visual de filtros digitales se pueden usar herramientas en línea como por ejemplo [MIcroModeler](https://www.micromodeler.com/dsp/#)


In [ ]:
# COMPLETAR  Completar el código de diseño del filtro pasa-altos de Butterworth



In [ ]:
# COMPLETAR Mostrar respuesta en frecuencia del filtro pasa-altos

In [ ]:
# COMPLETAR: Mostrar la ubicación de ceros y polos del filtro en el plano complejo 

#### 4.1.2 Filtrar los datos 

Para todos los sujetos de la base, filtrar los datos de aceleración __acc_x__, __acc_y__ y __acc_z__  para obtener __body_acc_x__, __body_acc_y__ y __body_acc_z__

Los datos filtrados se agregarán como nuevas columnas __body_acc_x__, __body_acc_y__ y __body_acc_z__ al DataFrame original.

> Elegir adecuadamente la forma de filtrar para no introducir un desfasaje con respecto a los datos originales.

In [ ]:
# COMPLETAR 
# Función que filtra una señal

def butter_highpass_filter(signal, fs, cutoff=0.2, order=2):
    """
    Aplica un filtro Butterworth pasa-altos a la señal de entrada.

    Parámetros:
        signal (array-like): Señal de entrada a filtrar.
        fs (float): Frecuencia de muestreo en Hz.
        cutoff (float): Frecuencia de corte en Hz (por defecto 0.2 Hz).
        order (int): Orden del filtro Butterworth (por defecto 2).

    Retorna:
        filtered_signal (np.ndarray): La señal filtrada.
    """
    filtered_signal = None
    #COMPLETAR


    return filtered_signal

In [ ]:
# COMPLETAR
# Recorrer todos los sujetos del DataFrame, 
# filtrar los datos de aceleración para los 3 ejes 
# y obtener la aceleración corporal (body acceleration). 
# Agregar las columnas body_acc_x, body_acc_y y body_acc_z al DataFrame original.





In [ ]:
# COMPLETAR: 
# Mostrar datos de aceleración originales y filtrados para un sujeto
# Verificar el resultado haciendo zoom en distintas actividades




## 5. Cálculo de características



Para determinar la actividad que está realizando una persona en cada momento, es neceario mirar las señales de acelerómetro en ventanas cortas de tiempo. El largo de ventana a considerar debe ser acorde al problema. Una ventana muy grande puede ser poco precisa temporalmente y abarcar distintas actividades, mientras que una ventana demasiado pequeña puede impedir entender bien las señales para determinar correctamente la actividad. 

En nuestro caso ventanas de 2.5s a 5s podrían ser adecuadas.  

En esta parte de la práctica vamos a agrupar los datos de las series temporales por ventanas de tiempo para posteriormente calcular algunos números en esas ventanas. Estos números, que llamamos __características__ (__features__ en inglés), esperamos que sean distintivos para las actividades y por lo tanto nos permitan discriminar a qué actividad pertenece una ventana de tiempo que estemos analizando.


### 5.1 Cálculo de características por ventanas

A partir del DataFrame __df_har__ queremos obtener un nuevo DataFrame __df_har_features_labels__ donde cada fila tenga las características calculadas y la etiqueta de actividad correspondiente.

Para esto tenemos una función que recibe las señales de los ejes y calcula características y etiquetas por ventanas  
> Nota: Esta función se va a ejecutar sin tener en cuenta la separación en sujetos. Esto no es un problema grave ya que las ventanas serán cortas. Además, como una ventana podría abarcar más de una actividad, se tomará como etiqueta para la ventana, la actividad preponderante en tiempo, en la ventana. 

In [ ]:
def calcular_caracteristicas_por_ventanas(signal_x, signal_y, signal_z, labels, fs=50, window_s=4, overlap=0.5):
    """
    Calcula características sobre las señales de acelerómetro en ventanas solapadas.

    Parámetros:
        signal_x, signal_y, signal_z: arrays 1D de aceleración en cada eje
        labels: array 1D de etiquetas (misma longitud que las señales)
        fs: frecuencia de muestreo (Hz)
        window_s: duración de la ventana en segundos
        overlap: fracción de solapamiento entre ventanas (0 a 1)

    Devuelve:
        features_array: array 2D (ventanas x características)
        labels_array: array 1D de etiquetas por ventana (etiqueta preponderante en la ventana)
    """
    n = len(signal_x)
    window_samples = int(window_s * fs)
    step = int(window_samples * (1 - overlap))
    features = []
    labels_window = []

    for start in range(0, n - window_samples + 1, step):
        end = start + window_samples
        x_win = signal_x[start:end]
        y_win = signal_y[start:end]
        z_win = signal_z[start:end]
        label_win = labels[start:end]
        # Características: energía por eje, magnitud media, desviación estándar
        energy_x = np.sum(x_win**2)
        energy_y = np.sum(y_win**2)
        energy_z = np.sum(z_win**2)
        mag = np.sqrt(x_win**2 + y_win**2 + z_win**2)
        mean_mag = np.mean(mag)
        std_mag = np.std(mag)
        
        features.append([
            energy_x, energy_y, energy_z,
            mean_mag, std_mag,
        ])
        # Etiqueta preponderante
        labels_window.append(np.bincount(label_win).argmax())

    features_array = np.array(features)
    labels_array = np.array(labels_window)
    return features_array, labels_array

#### 5.1.1 Calcular las características sobre las señales 
Calcular para __body_acc_x__, __body_acc_y__ y __body_acc_z__ 

In [ ]:
# Calcular características en ventanas de 4 segundos con 50% de solapamiento
ventana_s = 4
solapamiento = 0.5

body_acc_x = df_har.body_acc_x.values
body_acc_y = df_har.body_acc_y.values
body_acc_z = df_har.body_acc_z.values
labels = df_har.label.values


win_features, win_labels = calcular_caracteristicas_por_ventanas(
    body_acc_x, body_acc_y, body_acc_z,
    labels,
    fs=fs, window_s=ventana_s, overlap=solapamiento
)   

# Convertir a DataFrame
df_har_features_labels = pd.DataFrame(win_features, columns=[
    'energy_x', 'energy_y', 'energy_z',
    'mean_mag', 'std_mag',
  
])
# Agregar la columna de etiquetas
df_har_features_labels['label'] = win_labels

### 5.2 Espacio de características

Para cada ventana calculamos  **D** características y le asignamos una etiqueta de actividad. Si calculamos **N** ventanas, tenemos: 

$$
\text{Características   }~X \in \mathbb{R}^{N \times D} 
$$

$$
\text{Etiquetas   }~y \in \mathbb{R}^{N \times 1} 
$$

Cada ventana estará representada entonces por un punto del espacio $\mathbb{R}^{D}$. 

Nos interesa investigar si esos **D** números calculados para cada ventana permiten discriminar las distintas actividades. Más formalmente, si estos puntos en el espacio $\mathbb{R}^{D}$ son separables en regiones de forma que quedan agrupados por actividad.

Como eso es difícil de ver en muchas dimensiones, podemos empezar por ver para una o dos características.

#### 5.2.1 Visualizar distribución de valores para una característica mediante BoxPlots

Ver como se distribuyen los valores de una determinada característica para las distintas actividades.

* Estudiar qué es un boxplot
* Realizar una gráfica que incluya los BoxPlots para la característica __mean_mag__. La gráfica tendrá un boxplot por cada actividad.

Ver por ejemplo en matplotlib la [función boxplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html),  y [ejemplos](https://matplotlib.org/stable/gallery/statistics/boxplot_demo.html#sphx-glr-gallery-statistics-boxplot-demo-py) 



In [ ]:
# COMPLETAR 



#### 5.2.2 Visualizar para dos características

Ver cada ventana como un punto en el plano __std_mag__ vs. __mean_mag__.  
Colorear los puntos por actividad.


In [ ]:
# COMPLETAR Gráfica del plano std_mag vs. mean_mag




#### 5.2.3 Pairplot

Para poder repetir lo anterior y observar dos a dos todas las características se puede utilizar un **Pair plot**

[Ejemplo de pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) 




In [ ]:
# COMPLETAR Pairplot de todas las características por actividad





## 6. Clasificación - primeras ideas



* En base a los gráficos realizados en la parte 5, 
    * ¿cuáles parecen ser las características más relevantes?
    * ¿le parece posible discriminar las actividades en base a las características calculadas?
    * ¿qué nos permite discriminar el conjunto de características calculado?
      
* ¿Qué otras características se podrían extraer de las señales que aporten a la clasificación de actividades?

**RESPONDER**


Respuestas:
